In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from datetime import datetime,date,timedelta
import pickle

In [2]:
now = datetime.now().date()
sdate= datetime.now().date()-timedelta(days=365*2)
datediff = (now-sdate).days

In [3]:
allwind = pd.read_pickle('brommer_raw_2020-10-04_2022-10-03.pkl')

In [4]:
allwind['Month'] = pd.DatetimeIndex(allwind['Date']).month_name().str[:3]

In [14]:
allwind.loc[(allwind.Time.str.startswith(("10:","11:")) & allwind.Time.str.endswith("PM")) |\
            (allwind.Time.str.startswith(("12:","1:","2:","3:","4:","5:")) & allwind.Time.str.endswith("AM")) ,\
            "PartOfDay"] = "Overnight"

allwind.loc[allwind.Time.str.startswith(("6:","7:","8:","9:")) & \
            allwind.Time.str.endswith("AM"), \
            "PartOfDay"] = "Morning"

allwind.loc[(allwind.Time.str.startswith(("10:","11:")) & allwind.Time.str.endswith("AM")) |\
            (allwind.Time.str.startswith(("12:","1:")) & allwind.Time.str.endswith("PM")) ,\
            "PartOfDay"] = "Midday"

allwind.loc[allwind.Time.str.startswith(("2:","3:","4:","5:")) & \
            allwind.Time.str.endswith("PM"), \
            "PartOfDay"] = "Afternoon"

allwind.loc[allwind.Time.str.startswith(("6:","7:","8:","9:")) & \
            allwind.Time.str.endswith("PM"), \
            "PartOfDay"] = "Evening"

In [28]:
allwind.loc[allwind.Month.isin(['Dec','Jan','Feb']) ,\
            "Season"] = "Winter"

allwind.loc[allwind.Month.isin(['Mar','Apr','May']) ,\
            "Season"] = "Spring"
             
allwind.loc[allwind.Month.isin(['Jun','Jul','Aug']) ,\
            "Season"] = "Summer"
             
allwind.loc[allwind.Month.isin(['Sep','Oct','Nov']) ,\
            "Season"] = "Fall"

In [31]:
allwind[::len(allwind)//10]

,Date,Time,Season,Month,Daypart,Temp,Wind,Orient,Speed,Gust
0,2020-10-04,12:04 AM,Fall,Oct,Overnight,55.6,East,90.0,0.0,0.0
6,2020-12-14,12:34 AM,Winter,Dec,Overnight,45.4,WSW,247.5,0.0,0.9
4,2021-02-23,12:24 AM,Winter,Feb,Overnight,49.4,WSW,247.5,0.0,0.0
88,2021-05-20,7:24 AM,Spring,May,Morning,47.6,NE,45.0,1.2,4.0
161,2021-07-30,1:29 PM,Summer,Jul,Midday,63.9,SW,225.0,4.3,8.0
197,2021-10-09,4:29 PM,Fall,Oct,Afternoon,67.8,WSW,247.5,3.0,8.0
262,2021-12-19,9:52 PM,Winter,Dec,Evening,51.1,SSW,202.5,7.0,8.0
242,2022-03-01,8:14 PM,Spring,Mar,Evening,54.9,East,90.0,0.9,6.0
4,2022-05-12,12:24 AM,Spring,May,Overnight,47.7,NNE,22.5,2.4,4.0
3,2022-07-23,12:19 AM,Summer,Jul,Overnight,56.2,West,270.0,0.9,3.0


In [30]:
allwind = allwind[['Date','Time','Season','Month','Daypart','Temp','Wind','Orient','Speed','Gust']]

In [32]:
allwind.to_csv(f'brommer_raw_{sdate}_{now-timedelta(days=1)}.csv')
allwind.to_pickle(f'brommer_raw_coverage_{sdate}_{now-timedelta(days=1)}.pkl')

In [ ]:
ordermth=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
allwind.groupby('Month')['Orient'].mean()

In [ ]:
print(pd.DataFrame([dict(sorted(allwind.groupby('Month')['Speed'].mean().round(2).items(), key=lambda x: x[1]))]))
print(pd.DataFrame([dict(sorted(allwind.groupby('Month')['Orient'].mean().round(2).items(), key=lambda x: x[1]))]))

In [ ]:
monthdirs = (allwind.groupby('Month')['Orient'].mean()[ordermth]//22.5)*22.5
print(monthdirs)

In [ ]:
ordir={'North': 0.0,
 'NNE': 22.5,
 'NE': 45.0,
 'ENE': 67.5,
 'East': 90.0,
 'ESE': 112.5,
 'SE': 135.0,
 'SSE': 157.5,
 'South': 180,
 'SSW': 202.5,
 'SW': 225.0,
 'WSW': 247.5,
 'West': 270.0,
 'WNW': 292.5,
 'NW': 315.0,
 'NNW': 337.5}

In [ ]:
revord = dict([(value, key) for key, value in ordir.items()])
revord

In [ ]:
monthdirs.map(revord)

In [ ]:
print("chronological average cardinal wind orientation (not weighted by speed)")
revord = dict([(value, key) for key, value in ordir.items()])
print(monthdirs.map(revord))

print("chronological average wind direction (not weighted by speed)")
monthavgdir = allwind.groupby('Month')['Orient'].mean().astype(int)[ordermth]
print(monthavgdir)

print("chronological average closest cardinal wind direction (not weighted by speed)")
monthdirs = (allwind.groupby('Month')['Orient'].mean()[ordermth]//22.5)*22.5
print(monthdirs)

print("ascending monthly max wind speed")
dict(sorted(allwind.groupby('Month')['Speed'].max().round(2).items(), key=lambda x: x[1]))

print("ascending monthly max wind gust speed")
print(pd.DataFrame([dict(sorted(allwind.groupby('Month')['Gust'].max().items(), key=lambda x: x[1]))]))

print("ascending monthly max wind speed")
print(pd.DataFrame([dict(sorted(allwind.groupby('Month')['Speed'].mean().round(2).items(), key=lambda x: x[1]))]))

print("chronological monthly average wind speed")
mnthavgv = allwind.groupby('Month')['Speed'].mean().round(2)[ordermth]
print(pd.DataFrame([mnthavgv]))

print("ascending monthly average wind speed")
smnthavgv = [dict(sorted(allwind.groupby('Month')['Speed'].mean().round(2).items(), key=lambda x: x[1]))]
print(pd.DataFrame(smnthavgv))

In [ ]:
pd.DataFrame([dict(sorted(allwind.groupby('Month')['Orient'].mean().round(2).items(), key=lambda x: x[1]))])